# Verse Interpreter development

Essential installs:
- `pip3 install .....`


This version is used to test the first steps for the verse interpreter. The final version will be used as a full python file.

# ERROR-TYPE ENUMERATION

In [335]:
from enum import Enum

class ErrorType(Enum):
    SyntaxError = 'Wrong Syntax at'
    SemanticError = 'Wrong Semantics at'
    UnkownError = 'Operation Failure'

# LOGGER CLASSS

In [336]:
class Logger:
    def __init__(self):{}
        
    def __log__(self, string:str):{}

    def __log_error__(self,string:str, type:ErrorType):{}

In [337]:
class Console_Logger(Logger):

    def __log__(self, string:str):
        print(string)

    def __log_error__(self,string:str, type:ErrorType):       
        print("ERROR| " + type.value + ": " + string)


In [338]:
from enum import Enum
import string

# class syntax

class TokenTypes(Enum):
    # Data
    Integer = int
    Identifier = string #Names/Variables
    Type = None
    # Aritmetics
    Plus = "+"
    Minus = "-"
    Multiply = "*"
    Divide = "/"
    Greater = ">"
    GreaterEq = ">="
    Lower = "<"
    LowerEq = "<="
    Choice = "|"
    # Mehtods
    For = "for"
    LBracket = "("
    RBracket = ")"
    # Else
    EOF = None
    Colon = ":"
    Comma=","
    SemiColon =";"
    Binding =":="

    



# File Reader

In [339]:
class FileReader:
    def __init__(self):{}

    
    def get_Lines(self, name:str):
       
        try:
            f = open('..\modules\{}'.format(name),'r')
            lines = f.read().split("\n")
            return (lines,True)
        except:
            return ([],False)
        
       

   
reader = FileReader()
segments,read_success = reader.get_Lines('example.txt')
print(segments)

['halloo wie geht', 'ws', 'efaf', ' aefe ', ' eeerwre']


In [340]:
class Token:
     def __init__(self, type: TokenTypes,value, start_pos:int, end_pos:int):
        self.value = value
        self.type = type
        self.start_pos = start_pos
        self.end_pos = end_pos

     def __info__(self):
         return "{}:{}".format(self.type, self.value)


In [341]:
import string


class lexicon:
    def __init__(self, input: string):
        self.input = input
        self.index = 0
        self.current_char = self.input[self.index]

    def error():
        raise Exception('Invalid character.')
    
    # moves the pointer a character forward
    def forward(self, index) -> tuple[str,int]:
        index += 1

        # check if index out of range
        if (index >= len(self.input)):
            return None, index
        
        return self.input[index],index
    
    def future_token(self, index) -> tuple[Token,int]:
        index += 1

        # check if index out of range
        if (index >= len(self.input)):
            return None,index
        
        return self.get_token(self.input[index], index)
    
    def get_int(self, index: int) -> tuple[int,int]:
        result = ''

        # checks if there are multiple digits
        while True:
            if index < len(self.input) and self.input[index] != None and self.input[index].isdigit():
                result += self.input[index]

                if index + 1 < len(self.input) and self.input[index + 1].isdigit():
                    _,index = self.forward(index)
                else:
                    break
            else:
                break

        return int(result),index
    
    def get_token(self, char: string, index: int) -> tuple[Token,int]:
        if char is None:
            return Token(TokenTypes.EOF, None,0,0),index

        if char.isdigit():
            result,index = self.get_int(index)
            return Token(TokenTypes.Integer, result,0,0),index

        if char == TokenTypes.Divide.value:
            return Token(TokenTypes.Divide, TokenTypes.Divide.value,0,0),index
        
        if char == TokenTypes.Multiply.value:
            return Token(TokenTypes.Multiply, TokenTypes.Multiply.value,0,0),index
        
        if char == TokenTypes.Plus.value:
            return Token(TokenTypes.Plus, TokenTypes.Plus.value,0,0),index
        
        if char == TokenTypes.Minus.value:
            return Token(TokenTypes.Minus, TokenTypes.Minus.value,0,0),index
        
        if char == TokenTypes.SemiColon.value:
            return Token(TokenTypes.SemiColon, TokenTypes.SemiColon.value,0,0),index
        
        if char == TokenTypes.Comma.value:
            return Token(TokenTypes.Comma, TokenTypes.Comma.value,0,0),index
        
        if char == TokenTypes.Colon.value:
            return Token(TokenTypes.Colon, TokenTypes.Colon.value,0,0),index
        
        if char == TokenTypes.LBracket.value:
            return Token(TokenTypes.LBracket, TokenTypes.LBracket.value,0,0),index
        
        if char == TokenTypes.RBracket.value:
            return Token(TokenTypes.RBracket, TokenTypes.RBracket.value,0,0),index
        
        if char == ' ':
            char,index = self.forward(index)
            return self.get_token(char, index)
        
        return Token(TokenTypes.EOF, None,0,0),index

In [342]:
lexer = lexicon("723 + 2 - 5 * 1235 / 100")

while lexer.current_char is not None:
    token,lexer.index = lexer.get_token(lexer.current_char, lexer.index)
    print(str(token.value) + " is of the tokentype: " + str(token.type))
    future_token,index = lexer.future_token(lexer.index)
    if future_token is not None:
        print("Future:" + str(future_token.type) + "\n")
    lexer.current_char, lexer.index = lexer.forward(lexer.index)

723 is of the tokentype: TokenTypes.Integer
Future:TokenTypes.Plus

+ is of the tokentype: TokenTypes.Plus
Future:TokenTypes.Integer

2 is of the tokentype: TokenTypes.Integer
Future:TokenTypes.Minus

- is of the tokentype: TokenTypes.Minus
Future:TokenTypes.Integer

5 is of the tokentype: TokenTypes.Integer
Future:TokenTypes.Multiply

* is of the tokentype: TokenTypes.Multiply
Future:TokenTypes.Integer

1235 is of the tokentype: TokenTypes.Integer
Future:TokenTypes.Divide

/ is of the tokentype: TokenTypes.Divide
Future:TokenTypes.Integer

100 is of the tokentype: TokenTypes.Integer


In [343]:
class BaseNode:
    def __init__(self, token:Token) -> None:
        self.token = token

class BlockNode(BaseNode):
    def __init__(self, node:list[BaseNode]) -> None:
        self.node = node

class Program(BaseNode):
    def __init__(self, node:list[BlockNode]) -> None:
        self.node = node


class BlockNode(BaseNode):
    def __init__(self, nodes:list[BaseNode]) -> None:
        self.nodes = nodes

class NumberNode(BaseNode):
    def __init__(self, token:Token) -> None:
        super().__init__(token)
        self.value = token.value

class OperatorNode(BaseNode):
    def __init__(self, token:Token, leftNode: BaseNode, rightNode: BaseNode) -> None:
        super().__init__(token)
        self.leftNode = leftNode
        self.rightNode = rightNode

class UnaryNode(BaseNode):
     def __init__(self, token:Token, node) -> None:
        super().__init__(token)
        self.node = node

class ScopeNode(BaseNode):
    def __init__(self, token:Token, nodes:list[BaseNode], type) -> None: #Change into Variable/IdentifierNode
        super().__init__(token)
        self.nodes = nodes
        self.type = type

In [349]:
#Class that takes a parsed node, containes information if node could have been parsed
class ParsedNode:
    def __init__(self, node:BaseNode, hasSyntaxError:bool ):
        self.node = node
        self.hasSyntaxError = hasSyntaxError

    
        

In [346]:
class Interpreter:
    def __init__(self, parser):
        self.parser = parser

    def Ovisitor(self, node: OperatorNode):
        if node.token.type == TokenTypes.Plus:
            return self.visit(node.leftNode) + self.visit(node.rightNode)
        elif node.token.type == TokenTypes.Minus:
            return self.visit(node.leftNode) - self.visit(node.rightNode)
        elif node.token.type == TokenTypes.Multiply:
            return self.visit(node.leftNode) * self.visit(node.rightNode)
        elif node.token.type == TokenTypes.Divide:
            return self.visit(node.leftNode) // self.visit(node.rightNode)

    def Nvisitor(self, node: NumberNode):
        return node.value

    def interpret(self):
        tree = self.parser.parse()
        return self.visit(tree)
    
    def visit(self, node: BaseNode):
        if type(node) == OperatorNode:
            return self.Ovisitor(node)
        
        return self.Nvisitor(node)

In [347]:
class Parser:
    def __init__(self, lexer: lexicon):
       self.logger: Logger = Console_Logger()
       self.end = False
       self.lexer = lexer
       self.current_token, self.index = lexer.get_token(
           self.lexer.current_char, self.lexer.index)
       

    def parse(self):     
        node = self.program().node
        return node
       
  

    def program(self):
        return self.expr()

    def block(self):
        pass

    def statement_list(self):
        pass

    def statement(self):
        pass

    def func_call(self):
        pass
    
    def func_call_param(self):
        pass

    def func_decl(self):
        pass 

    def func_dec_param(self):
        pass

    def if_statement(self):
        pass

    def for_loop(self):
        pass

    def nested_scope(self):
        pass


#####################################

    def expr(self):      
        return self.arith()

    def choice(self):
        pass 

    def eqaul(self):
        pass

    def gl(self):
        pass

    def arith(self):
        #term ((PLUS|MINUS) term)*

        left_node = self.term()

        while(self.current_token.type == TokenTypes.Plus or self.current_token.type == TokenTypes.Minus):
            token = self.current_token
            self.advance_to_next_token()
            right_node = self.term()
            if(right_node.hasSyntaxError):
                return right_node
            else:
                return ParsedNode(OperatorNode(token,left_node.node,right_node.node),False)
        return left_node

    def term(self):
        #factor ((MUL|DIV) factor)*

        left_node = self.factor()

        while(self.current_token.type == TokenTypes.Multiply or self.current_token.type == TokenTypes.Divide):
            token = self.current_token
            self.advance_to_next_token()
            right_node = self.factor()
            if(right_node.hasSyntaxError):
                return right_node
            else:
                return ParsedNode(OperatorNode(token,left_node.node,right_node.node),False)
        return left_node
    

    def factor(self):
        token = self.current_token
        index = self.index
      # INTEGER  
      # : tuple
      # : (MINUS|PLUS) arith
      # : func_call
      # : indexing

        if(token.type == TokenTypes.Integer):
            self.advance_to_next_token()
            return ParsedNode(NumberNode(token),False)
        
        if(self.current_token.type == TokenTypes.Plus or self.current_token.type == TokenTypes.Minus):
            self.advance_to_next_token()
            node = self.arith()
            return ParsedNode(UnaryNode(token,node),False)
        
        node = self.tuple(token)
        if(node.hasSyntaxError):
            self.set_to_token(index)
        else: return node
        
        return ParsedNode(None,True) #Returns invalid Node on invalid Syntax
        

    def binding(self):
        pass

    def scope(self):
        pass

    def Identifier(self):
        pass

    def type(self):
        pass 

    def tuple(self):
        #tuple: LB binding|expr (COMMA binding|expr)*? RB
        if(self.current_token.type == TokenTypes.LBracket):
            self.advance_to_next_token()
            node = self.arith()
            if(self.current_token.type == TokenTypes.RBracket):
                self.advance_to_next_token()
                return node
        return ParsedNode(None,True)
        

    def indexing(self):
        pass







    def advance_to_next_token(self):
        self.lexer.current_char, self.lexer.index = self.lexer.forward(
            self.index)
        self.current_token, self.index = lexer.get_token(
            self.lexer.current_char, self.lexer.index)

        if (self.current_token.type == TokenTypes.EOF):
            self.end = True
        print(self.current_token.__info__())

    def set_to_token(self,index): 
        self.index = index
        self.advance_to_next_token()




    



In [348]:


text = "(2 * 3) * (2 + 2)"
lexer = lexicon(text)
parser = Parser1(lexer)
interpreter = Interpreter(parser)
result = interpreter.interpret()
result



TokenTypes.LBracket:(
TokenTypes.Integer:2
TokenTypes.Multiply:*
TokenTypes.Integer:3
TokenTypes.RBracket:)
TokenTypes.Multiply:*
TokenTypes.LBracket:(
TokenTypes.Integer:2
TokenTypes.Plus:+
TokenTypes.Integer:2
TokenTypes.RBracket:)
TokenTypes.Type:None


24